In [4]:
import os
import json
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import time
from datetime import timedelta
import h5py
from tqdm import tqdm
import argparse
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt

import sys

ProjectPath = "/home/liranc6/ecg/ecg_forecasting" #os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd())))
sys.path.append(ProjectPath)  # Add the parent directory to the sys.path

import liran_project.utils.dataset_loader as dataset_loader

# import liran_project.train as liran_train
from liran_project.utils.dataset_loader import SingleLeadECGDatasetCrops
from liran_project.utils.util import modify_z_and_omega

sys.path.append('/home/liranc6/ecg/ecg_forecasting/SSSD_main')

from SSSD_main.src.utils.util import find_epoch, print_size, sampling, calc_diffusion_hyperparams
from SSSD_main.src.utils.util import get_mask_mnr, get_mask_bm, get_mask_rm, get_mask_pred

import SSSD_main.src.utils.util as util
from SSSD_main.src.imputers.DiffWaveImputer import DiffWaveImputer
from SSSD_main.src.imputers.SSSDSAImputer import SSSDSAImputer
from SSSD_main.src.imputers.SSSDS4Imputer import SSSDS4Imputer

from sklearn.metrics import mean_squared_error
from statistics import mean

ModuleNotFoundError: No module named 'liran_project'

In [1]:
file_path = \
"/home/liranc6/ecg_forecasting/liran_project/results/icentia11k/SSSDS4/context248_label1248/T200_beta00.0001_betaT0.02/best_model:_iter:_1_loss:_0.9853200713793436.pth"

In [6]:
# open file and get the keys
checkpoint = torch.load(file_path, map_location='cpu')
print(checkpoint.keys())
checkpoint["wandb_id"]

dict_keys(['model_state_dict', 'optimizer_state_dict', 'wandb_id'])


'kdnog64l'

In [13]:
dict_path = \
"/home/liranc6/ecg_forecasting/liran_project/results/icentia11k/SSSDS4/context3748_label2500/T200_beta00.0001_betaT0.02"

In [14]:
import torch

for root, dirs, files in os.walk(dict_path):
    found = False
    for file in files:
        if file.endswith(".pth"):
            file_path = os.path.join(root, file)
            model_checkpoint = torch.load(file_path)
            if "wandb_id" in model_checkpoint:
                print(model_checkpoint["wandb_id"])
                if model_checkpoint["wandb_id"] == "f7y4e71z":
                    print("found the file")
                    print(file_path)
                    found = True
                    break
                else:
                    #skip to next directory
                    break
                    
            else:
                print("not found")
    if found:
        break

212frl4y
4sd0opjk
